# tabulate sas results

In [1]:
import pandas as pd
import numpy as np
import os
import re

In [2]:
# os.chdir('SAS')

In [3]:
def sheet_names(filename):
    excel = f"{filename}"
    file = pd.ExcelFile(excel)
    sheet_name = file.sheet_names
    sheet_name_no_space = [x.replace(' ','') for x in sheet_name]
    sheet_name_no_space_no_dash = [x.replace('-','_') for x in sheet_name_no_space]
    return excel,sheet_name,sheet_name_no_space_no_dash

In [4]:
def import_tukey(filename,sheet_name):
    table = pd.read_excel(filename, sheet_name = sheet_name, header=[2]).columns[0]
    if (re.findall('Tukey-Kramer Grouping',table) == ['Tukey-Kramer Grouping'])|(re.findall('Conservative',table) == ['Conservative'])|(re.findall('Tukey Grouping',table) == ['Tukey Grouping']):
        df = pd.read_excel(filename, sheet_name = sheet_name, header=[4], index_col =[0]).reset_index()
        if (re.findall('Conservative',table) == ['Conservative']):
            note = df.iloc[len(df)-1,0].replace('\n',' ')
            note = re.findall('The LINES display does not reflect all significant comparisons. The following additional pairs are significantly different: (.*).',note)
            df['differences'] = ' '.join(note)
        else:
            df['differences'] = ''
        df = df.dropna(subset=['Estimate']).fillna('')
        unnamed_col_list = list(filter(re.compile('Unnamed').match,df.columns))
        groupings_col_list = [f'grouping_{n}' for n in range(1,len(unnamed_col_list)+1)]
        rename_dic = dict(zip(unnamed_col_list,groupings_col_list))
        df = df.rename(columns =rename_dic)
        df['letter_group'] = ''
        for g in groupings_col_list:
            df['letter_group'] = df['letter_group'] + df[g]
        df['letter_group'] = df['letter_group'].map(lambda x: ''.join(sorted(x)))
        df['letter_group'] = df['letter_group'].str.lower()
        df = df.drop(columns=list(filter(re.compile('grouping').match,df.columns)))
        return df

In [5]:
filename = 'abundance_and_incidence.xlsx'
excel,sheet_name,sheet_name_no_space_no_dash = sheet_names(filename)
df_tukey_concat = pd.DataFrame()
for name in sheet_name:
    df_tukey = import_tukey(filename,name)
    df_tukey_concat = pd.concat([df_tukey_concat,df_tukey])
df_tukey_concat.reset_index()

C:\Users\gtn49005\Anaconda3\envs\envi\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,index,variable,Estimate,differences,letter_group
0,0,Mi_a,4.69930,,a
1,2,Ma_a,2.59500,,b
2,4,MiMa_a,2.23740,,b
3,6,Mj_a,1.43680,,bc
4,8,MaMj_a,-0.09441,,cd
5,10,MiMaMj_a,-1.66500,,de
6,12,MiMj_a,-2.76360,,e
7,0,Mi_i,1.51510,,a
8,2,MiMa_i,-2.31250,,b
9,4,Ma_i,-2.86220,,b


In [6]:
def create_table(df,value,index,aggfunc):
    df = pd.pivot_table(df, 
                           values=value, 
                           index=index, aggfunc=aggfunc)
    df = df.reset_index().set_index('trt')
    return df

In [7]:
filename = 'survey_data_for_sas.xlsx'
df = pd.read_excel(filename)
df = df.iloc[:,20:]
df_mean = pd.DataFrame(df.mean()).reset_index()
df_sem = pd.DataFrame(df.sem()).reset_index()
df_max = pd.DataFrame(df.max()).reset_index()
df_min_non_zero = pd.DataFrame(df.mask(df==0).min()).reset_index()
df_min = pd.DataFrame(df.min()).reset_index()
df_merge = df_mean.merge(df_sem, on = 'index', how = 'inner')
df_merge = df_merge.merge(df_max, on = 'index', how = 'inner')
df_merge = df_merge.merge(df_min_non_zero, on = 'index', how = 'inner')
df_merge = df_merge.merge(df_min, on = 'index', how = 'inner')
df_merge.columns = ['variable','mean','sem','max','min_non_zero','min']
df_merge = df_merge.merge(df_tukey_concat, on = 'variable', how = 'inner')
df_merge = df_merge.drop(columns = ['Estimate','differences'])
df_merge.to_excel('mean_sem.xlsx', index=False)
df_merge

C:\Users\gtn49005\Anaconda3\envs\envi\lib\site-packages\pandas\core\frame.py:9186: FutureWarning: Passing 'suffixes' which cause duplicate columns {'0_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(


,variable,mean,sem,max,min_non_zero,min,letter_group
0,Mi_i,0.819820,0.036645,1,1.0,0,a
1,Ma_i,0.054054,0.021560,1,1.0,0,b
2,Mj_i,0.009009,0.009009,1,1.0,0,b
3,MiMa_i,0.090090,0.027299,1,1.0,0,b
4,MiMj_i,0.009009,0.009009,1,1.0,0,b
5,MaMj_i,0.009009,0.009009,1,1.0,0,b
6,MiMaMj_i,0.009009,0.009009,1,1.0,0,b
7,Mi_a,109.873874,22.767342,1520,1.0,0,a
8,Ma_a,13.396396,11.694369,1296,5.0,0,b
9,Mj_a,4.207207,4.207207,467,467.0,0,bc
